In [1]:
# neatness of code:
%load_ext pycodestyle_magic
%pycodestyle_on

In [5]:
# Packages:
import sqlite3
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
# First que list all objects in the database:
conn = sqlite3.connect('/home/ferran/Desktop/master/R&I/ml/final_project_skeleton/db.sqlite3')
pd.read_sql(""" SELECT * FROM sqlite_master;""", conn)
# load tables:
#summ_vars = pd.read_sql("""SELECT * FROM Indicators;""", conn)
long = pd.read_sql("""SELECT * FROM CountryIndicators;""", conn)
long

,CountryCode,IndicatorCode,Year,Value
0,ARB,SP.ADO.TFRT,1960,1.335609e+02
1,ARB,SP.POP.DPND,1960,8.779760e+01
2,ARB,SP.POP.DPND.OL,1960,6.634579e+00
3,ARB,SP.POP.DPND.YG,1960,8.102333e+01
4,ARB,MS.MIL.XPRT.KD,1960,3.000000e+06
...,...,...,...,...
5026216,ZWE,SP.URB.TOTL.IN.ZS,2010,3.319600e+01
5026217,ZWE,SP.URB.GROW,2010,1.279630e+00
5026218,ZWE,DT.DOD.DIMF.CD,2010,5.287290e+08
5026219,ZWE,SN.ITK.VITA.ZS,2010,4.930002e+01


In [ ]:
READ DATABASE

In [6]:
# we save the long database with a reshape: year x variables in datasets.txt
country = long[long['CountryCode'].str.contains("ARB")][["IndicatorCode","Year","Value"]]
df = country.pivot(index='Year', columns='IndicatorCode',values='Value')
df = df.fillna(method='ffill') # impute NA with the previous observation.
df['Country'] = "ARB"
for n in long.CountryCode.unique(): #long.CountryCode.unique()
    country2 = long[long['CountryCode'].str.contains(n)][["IndicatorCode","Year","Value"]]
    df2 = country2.pivot(index='Year', columns='IndicatorCode',values='Value')
    df2 = df2.fillna(method='ffill')
    df2['Country'] = n
    df = df.append(df2)
#np.unique(df.Country)
#df.shape
#np.savetxt('/home/ferran/Desktop/master/R&I/ml/final_project_skeleton/dataset.txt', result, delimiter='\t',fmt='%s')
#         header=
df.columns = df.columns.str.replace(".", "_")
#df = df.to_numpy()
#import pandas
#df = df.to_frame()
#complete = df.apply(lambda x: x.count(), axis=1) #complete cases per row.
aa = df.copy()


LINEAR MIXED MODEL

In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

# Creation of Yt-1 and demonstration Yt-2 is not significative.
df['lag1']=df['NY_GDP_MKTP_KD_ZG'].shift(1)
#df['lag2']=df['NY_GDP_MKTP_KD_ZG'].shift(2)
#df = df[df['NY_GDP_MKTP_KD_ZG'].notna()]
df['Time'] = df.index

df = df.dropna(subset=["NY_GDP_MKTP_KD_ZG","lag1"])
# Countries strings to numeric values:
groups = df[["Country"]].replace(pd.unique(df.Country), 
            list(range(0,len(pd.unique(df.Country)))))
groups = pd.to_numeric(groups.Country)

# subset of variables
df_sub = df[["NY_GDP_MKTP_KD_ZG","Country","lag1","Time"]]

# mixed linear model: group + year.
md = smf.mixedlm("NY_GDP_MKTP_KD_ZG ~ lag1", df_sub, groups=groups)#, re_formula="~Time" #df["Country"].to_numpy()
mod = md.fit()
print(mod.summary())
test = sms.jarque_bera(mod.resid)
#lzip(name, test)
# extraction of residuals
df['residuals'] = mod.resid
df['Country'] = groups




             Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: NY_GDP_MKTP_KD_ZG
No. Observations: 8964    Method:             REML             
No. Groups:       238     Scale:              36.1934          
Min. group size:  1       Log-Likelihood:     -28897.2537      
Max. group size:  67      Converged:          Yes              
Mean group size:  37.7                                         
-----------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|   [0.025   0.975]
-----------------------------------------------------------------
Intercept     2.882      0.104   27.628   0.000    2.677    3.086
lag1          0.281      0.011   26.405   0.000    0.260    0.302
Group Var     1.103      0.036                                   



REMOVE NA

In [12]:
aa =range(1,len(df))
df.shape # 1018
# Require that many non-NA values.
df.dropna(thresh=np.quantile(aa, 0.6), axis=1).shape # 238
df = df.fillna(method='bfill')
df = df.dropna(axis=1)
df

,AG_AGR_TRAC_NO,AG_CON_FERT_PT_ZS,AG_CON_FERT_ZS,AG_LND_AGRI_K2,AG_LND_AGRI_ZS,AG_LND_ARBL_HA_PC,AG_LND_ARBL_ZS,AG_LND_CREL_HA,AG_LND_CROP_ZS,AG_LND_EL5M_ZS,...,SE_XPD_CTER_ZS,SE_XPD_PRIM_ZS,SE_XPD_SECO_ZS,SE_XPD_MTER_ZS,SE_XPD_TERT_PC_ZS,SH_MLR_NETS_ZS,CM_MKT_INDX_ZG,lag1,Time,residuals
Year,,,,,,,,,,,,,,,,,,,,,
1978,207256.0,31.460708,62.501114,4305790.0,31.604469,0.322672,3.659207,23263857.0,0.321063,0.966807,...,96.696342,27.13887,16.158621,38.567478,36.957851,2.3,18.700001,8.298998,1978,-3.437871
1979,224684.0,31.460708,62.501114,4307280.0,31.615406,0.311510,3.651059,22086289.0,0.321579,0.966807,...,96.696342,27.13887,16.158621,38.567478,36.957851,2.3,18.700001,1.674562,1979,7.033302
1980,236194.0,31.460708,62.501114,4318030.0,31.694310,0.301061,3.644307,22988703.0,0.320548,0.966807,...,96.696342,27.13887,16.158621,38.567478,36.957851,2.3,18.700001,10.283988,1980,3.915767
1981,255349.0,31.460708,62.501114,4276396.0,31.388717,0.293647,3.668352,23489576.0,0.328646,0.966807,...,96.696342,27.13887,16.158621,38.567478,36.957851,2.3,18.700001,9.586066,1981,-1.273637
1982,271006.0,31.460708,62.501114,4279693.4,31.412920,0.286004,3.684464,22694791.0,0.340867,0.966807,...,96.696342,27.13887,16.158621,38.567478,36.957851,2.3,18.700001,4.200517,1982,-7.601283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,22496.0,210.442742,32.356341,163000.0,42.135195,0.312311,10.598423,2289312.0,0.258498,0.000000,...,94.774139,51.58009,25.604900,76.869751,61.999901,3.0,912.281279,-5.711084,2006,-3.999316
2007,22496.0,171.109177,27.035250,162000.0,41.876696,0.300802,10.339925,1949453.0,0.258498,0.000000,...,94.774139,51.58009,25.604900,76.869751,61.999901,3.0,-83.786962,-3.461495,2007,-4.823378
2008,22496.0,147.903481,21.994118,164500.0,42.522942,0.314921,10.986170,2235440.0,0.258498,0.000000,...,94.774139,51.58009,25.604900,76.869751,61.999901,3.0,-83.786962,-3.653327,2008,-18.785085


LASSO
With Lasso, the higher the alpha parameter, the fewer features selected.
The features with the highest absolute coef_ value are considered the most important

But in short, the mutual information between two variables, measures how much a given feature can explain another (target), or more technically, how much information about the target will variable will be obtained by having observed a feature.

The mutual information between two random variables is a non-negative value that measures the dependencies between variables. The mutual information between independent random variables is equal to 0, and the larger the mutual information, the stronger the dependency.
This function relies on the nonparametric method of entropy estimation based on k-nearest neighbor distance


In [48]:
#from sklearn.feature_selection import 
#from sklearn.linear_model import LassoCV

from sklearn.feature_selection import f_regression, mutual_info_regression

# input
covs = df.copy().drop(["NY_GDP_MKTP_KD_ZG","residuals"], 1)
Y = df[['residuals']]
info= feature_selection.mutual_info_regression(covs, np.ravel(Y))

# model
#clf = ElasticNetCV(tol=0.1,max_iter=3000).fit(covs, np.ravel(Y))
#importance = np.abs(clf.coef_)




# important features 
#feature_names = covs.columns
#idx_features = (-importance).argsort()[:sum(abs(importance)>0)]
#name_features = np.array(feature_names)[idx_features]
#print('Selected features: {}'.format(name_features))

In [61]:
#info[info>0.0].shape
d = {'name': covs.columns, 'weight': info}
final_df =pd.DataFrame(data=d)
final_df[["weight"]]


final_df.sort_values(by="weight",ascending=False)[1:48]
summ_vars = pd.read_sql("""SELECT * FROM Indicators;""", conn)
summ_vars #
vars = ["NY.GNP.MKTP.KD.ZG"]
df2 = summ_vars.loc[summ_vars["IndicatorCode"].isin(vars)]
df2
#summ_vars["IndicatorCode"].isin(vars)

#long[["IndicatorCode"]]
#vars = ["SP.POP.DPND", "MS.MIL.XPRT.KD", "SP.URB.GROW"]
#df2 = long.loc[long["IndicatorCode"].isin(vars)]

,IndicatorCode,Topic,IndicatorName,ShortDefinition,LongDefinition,UnitOfMeasure,Periodicity,BasePeriod,OtherNotes,AggregationMethod,LimitationsAndExceptions,NotesFromOriginalSource,GeneralComments,Source,StatisticalConceptAndMethodology,DevelopmentRelevance,RelatedSourceLinks,OtherWebLinks,RelatedIndicators,LicenseType
306,NY.GNP.MKTP.KD.ZG,Economic Policy & Debt: National accounts: Growth rates:,GNI growth (annual %),,GNI (formerly GNP) is the sum of value added by all resident producers plus any product taxes (less subsidies) not included in the valuation of output plus net receipts of primary income (compensation of employees and property income) from abroad.,,Annual,,,Weighted average,,,,"World Bank national accounts data, and OECD National Accounts data files.",,,,,,Open


CORRELATIONS

In [428]:
#df[[name_features]]
name_features

array(['NE_CON_PETC_KD', 'TX_VAL_MRCH_CD_WT', 'NE_CON_GOVT_CD',
       'NE_GDI_TOTL_KD', 'NE_CON_PETC_CD', 'NY_GDP_FCST_KD',
       'NY_GNP_MKTP_CD', 'NY_GDP_FCST_CD', 'NY_GDY_TOTL_KD',
       'NE_CON_PRVT_PP_CD', 'NV_SRV_TETC_CD', 'NY_GDP_MKTP_KD',
       'BX_GSR_GNFS_CD', 'CM_MKT_TRAD_CD', 'NE_CON_TOTL_KD',
       'NE_CON_TETC_KD', 'NE_DAB_TOTL_CD', 'NY_GNP_MKTP_KD',
       'NY_GDP_MKTP_PP_KD', 'NE_CON_TETC_CD', 'NE_CON_PRVT_KD',
       'NY_GDS_TOTL_CD', 'NY_GNP_MKTP_PP_KD', 'NE_DAB_TOTL_KD',
       'BX_GSR_TOTL_CD', 'NE_EXP_GNFS_KD', 'NY_ADJ_NNTY_KD'], dtype=object)

In [6]:
#### 1) We find Y: EstimatedGDPGrowth for the next year:
#summ_vars2 = summ_vars[summ_vars['IndicatorName'].str.contains("GDP")]
#summ_vars2[~summ_vars2.IndicatorName.str.contains("% of GDP")]
    # Response variable: GDP growth (annual %): NY.GDP.MKTP.KD.ZG
    # GPD covariate: GDP (constant 2005 US$), NY.GDP.MKTP.KD [look description of response if any doubt].
#long.CountryCode.unique()[1:10]

ERROR! Session/line number was not unique in database. History logging moved to new session 41
7.18.1


1:1: E266 too many leading '#' for block comment
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
4:5: E116 unexpected indentation (comment)
5:5: E116 unexpected indentation (comment)
5:80: E501 line too long (105 > 79 characters)
6:1: E265 block comment should start with '# '
7:11: E225 missing whitespace around operator


In [20]:
#### 3) NA & corr analysis: df: 629 rows x 51 columns.
ncol = len(df.columns)
complete = df.apply(lambda x: x.count(), axis=1) #complete cases per row.
    # complete.quantile(np.arange(0.0, 1.1, 0.1))
# We keep 80% complete cases: 51*0.8 =40 
df2 = df[complete>40] # 184 rows x 51 columns
    # CAREFUL: For some countries de response has a lot of NA, we should check this.
mat_corr =df2.T.corr() # transpose matrix, then correlation.
corrY_sort= mat_corr['NY.GDP.MKTP.KD.ZG'].iloc[np.flip(mat_corr['NY.GDP.MKTP.KD.ZG'].abs().argsort())]
    # length of corrY_sort: 241
high_corr=corrY_sort[abs(corrY_sort) > 0.7] # abreviation & correlation.
translation_high_corr = summ_vars[summ_vars['IndicatorCode'].str.contains('|'.join(high_corr.index.tolist()))]
translation_high_corr

,IndicatorCode,Topic,IndicatorName,ShortDefinition,LongDefinition,UnitOfMeasure,Periodicity,BasePeriod,OtherNotes,AggregationMethod,LimitationsAndExceptions,NotesFromOriginalSource,GeneralComments,Source,StatisticalConceptAndMethodology,DevelopmentRelevance,RelatedSourceLinks,OtherWebLinks,RelatedIndicators,LicenseType
292,NE.EXP.GNFS.KD.ZG,Economic Policy & Debt: National accounts: Growth rates,Exports of goods and services (annual % growth),,"Annual growth rate of exports of goods and services based on constant local currency. Aggregates are based on constant 2005 U.S. dollars. Exports of goods and services represent the value of all goods and other market services provided to the rest of the world. They include the value of merchandise, freight, insurance, transport, travel, royalties, license fees, and other services, such as communication, construction, financial, information, business, personal, and government services. They exclude compensation of employees and investment income (formerly called factor services) and transfer payments.",,Annual,,,Weighted average,,,,"World Bank national accounts data, and OECD National Accounts data files.",,,,,,Open
294,NY.GDP.MKTP.KD.ZG,Economic Policy & Debt: National accounts: Growth rates,GDP growth (annual %),,Annual percentage growth rate of GDP at market prices based on constant local currency. Aggregates are based on constant 2005 U.S. dollars. GDP is the sum of gross value added by all resident producers in the economy plus any product taxes and minus any subsidies not included in the value of the products. It is calculated without making deductions for depreciation of fabricated assets or for depletion and degradation of natural resources.,,Annual,,,Weighted average,"Each industry's contribution to growth in the economy's output is measured by growth in the industry's value added. In principle, value added in constant prices can be estimated by measuring the quantity of goods and services produced in a period, valuing them at an agreed set of base year prices, and subtracting the cost of intermediate inputs, also in constant prices. This double-deflation method requires detailed information on the structure of prices of inputs and outputs.\n\nIn many industries, however, value added is extrapolated from the base year using single volume indexes of outputs or, less commonly, inputs. Particularly in the services industries, including most of government, value added in constant prices is often imputed from labor inputs, such as real wages or number of employees. In the absence of well defined measures of output, measuring the growth of services remains difficult.\n\nMoreover, technical progress can lead to improvements in production processes and in the quality of goods and services that, if not properly accounted for, can distort measures of value added and thus of growth. When inputs are used to estimate output, as for nonmarket services, unmeasured technical progress leads to underestimates of the volume of output. Similarly, unmeasured improvements in quality lead to underestimates of the value of output and value added. The result can be underestimates of growth and productivity improvement and overestimates of inflation.\n\nInformal economic activities pose a particular measurement problem, especially in developing countries, where much economic activity is unrecorded. A complete picture of the economy requires estimating household outputs produced for home use, sales in informal markets, barter exchanges, and illicit or deliberately unreported activities. The consistency and completeness of such estimates depend on the skill and methods of the compiling statisticians.\n\nRebasing of national accounts can alter the measured growth rate of an economy and lead to breaks in series that affect the consistency of data over time. When countries rebase their national accounts, they update the weights assigned to various components to better reflect current patterns of production o

in many real applications variables are structured into groups, and group sparsity rather than individual sparsity is desired. 
group LASSO: solutions that are both between and within group sparse.
This way, you obtain solutions that are sparse, meaning that many of the β coefficients will be sent to 0 and your model will make predictions based on the few coefficients that are not 0.

The major advantage of these methods compared to OLS is that they can handle multicollinearity, i.e. a predictor matrix with rank less than the number of its columns.

Another thing to remember is that neither Ridge nor Lasso actually respond well to outlying observations.

LASSO is not a type of model. It is a method for selecting variables and shrinking coefficients to adjust for complexity.

The assumptions are those of the type of model it is applied to, which could be ordinary least squares regression, logistic regression

In [ ]:
#### Helpful comments:
# I want to predict year 2010 bc is what we have.
# we must include in the model Y_{t-1}.
# First we should do feature selection (lasso), then NA imputation, then prediction (quantile regression or gradient boosting.
num_models, k_folds = error.shape
# error is a matrix of shape (number_of_models, k_folds)
print(f'We are considering a grid of {num_models} models, optimized based on {k_folds}-folds cross validation')

# Obtain the mean error across different folds
error = np.mean(error, axis=1)


# Select the minimum error
minimum_error_idx = np.argmin(error)

# Select the parameters associated to mininum error values
optimal_parameters = cv_class.retrieve_parameters_value(minimum_error_idx)
optimal_lambda = optimal_parameters.get('lambda1')
optimal_alpha = optimal_parameters.get('alpha')

# Define asgl class using optimal values
asgl_model = asgl.ASGL(model=model, penalization=penalization, lambda1=optimal_lambda, alpha=optimal_alpha)


MOdel: Adaptive sparse group lasso
Loss function: QRE
    If you are interested in the mean, use MSE, if in the median, use quantile. 
    One big difference is that the mean is more affected by outliers and other extreme data.
The main appeal of quantile regression is that it is more robust than OLS. The downside is that if all assumptions are met, it will be less efficient

This is just a Markdown, now changed